# Connection Matrices

This notebook outlines the method used for constructing the matrices describing how the matrices should be constructed from the nested model structure.

In [1]:
from dataclasses import dataclass
from functools import reduce
from typing import Final
import operator
import numpy as np
from numpy.typing import NDArray, ArrayLike
from model import Zone, Layer, Hillslope, Model

In [2]:
hs: Hillslope = Hillslope([
    Layer([ Zone(), Zone()]),
    Layer([Zone()]),
    Layer([Zone()])
])
model: Model = Model([hs])
len(model)

4

In [3]:
def get_hillslope_vert_mat(hs: Hillslope) -> NDArray:
    n: Final[int] = len(hs)
    mat: NDArray = np.zeros((n, n), dtype=float)
    cz: int = 0 # The current zone

    if len(hs) == 0:
        raise ValueError("Hillslope must have at least one layer")

    rect_domain: NDArray = np.zeros((len(hs.layers), len(hs[0])), dtype=int)
    for i, ly in enumerate(hs):
        if len(ly) == 1:
            rect_domain[i, :] = cz
            cz += 1
        elif len(ly) == rect_domain.shape[1]:
            for j, _ in enumerate(ly):
                rect_domain[i, j] = cz
                cz += 1
        else:
            raise ValueError("Invalid model structure encountered")

    cz = 0
    layer: Layer
    for i, layer in enumerate(hs):
        match hs[i+1]:
            case Layer():
                for j, _ in enumerate(layer):
                    mat[cz, rect_domain[i+1, j]] = 1.0
                    cz += 1
            case None:
                cz += len(layer)
                continue
            case _:
                raise ValueError("Invalid model structure")
        
    return mat.T

def get_vert_mat(model: Model) -> NDArray:
    hs_blocks: list[NDArray] = [get_hillslope_vert_mat(hs) for hs in model]
    model_dim = len(model)
    mat: NDArray = np.zeros((model_dim, model_dim))

    cur: int = 0
    for b in hs_blocks:
        mat[cur:cur+b.shape[0], cur:cur+b.shape[0]] = b 

    return mat

def get_hillslope_lat_mat(hs: Hillslope) -> NDArray:
    n: Final[int] = len(hs)
    mat: NDArray = np.zeros((n, n), dtype=float)
    cz: int = 0 # The current zone

    if len(hs) == 0:
        raise ValueError("Hillslope must have at least one layer")

    rect_domain: NDArray = np.zeros((len(hs.layers), len(hs[0])), dtype=int)
    for i, ly in enumerate(hs):
        if len(ly) == 1:
            rect_domain[i, :] = cz
            cz += 1
        elif len(ly) == rect_domain.shape[1]:
            for j, _ in enumerate(ly):
                rect_domain[i, j] = cz
                cz += 1
        else:
            raise ValueError("Invalid model structure encountered")

    cz = 0
    layer: Layer
    for i, layer in enumerate(hs):
        for j, _ in enumerate(layer):
            match layer[j+1]:
                case Zone():
                    mat[cz, rect_domain[i, j+1]] = 1.0
                    cz += 1
                case None:
                    cz += 1
                    continue
                case _:
                    raise ValueError("Invalid model structure")
        
    return mat.T

def get_lat_mat(model: Model) -> NDArray:
    hs_blocks: list[NDArray] = [get_hillslope_lat_mat(hs) for hs in model]
    model_dim = len(model)
    mat: NDArray = np.zeros((model_dim, model_dim))

    cur: int = 0
    for b in hs_blocks:
        mat[cur:cur+b.shape[0], cur:cur+b.shape[0]] = b 

    return mat

In [4]:
lat = get_lat_mat(model)
vert = get_vert_mat(model)

print(f"Lat matrix: \n{lat}")
print(f"Vert matrix: \n{vert}")

Lat matrix: 
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Vert matrix: 
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 1. 0. 0.]
 [0. 0. 1. 0.]]


# Constructing the size matrix

The purpsoe of this function is to produce the matrix that describes which forcing data influence a single zone. For example, a hillslope with two snow zones, one snow zone, and one groundwater zone will have a connection matrix that looks like:

$$
\begin{bmatrix}
c_1 & 0 \\
0 & c_2
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
0 & 1 \\
1 & 1 \\
1 & 1 \\
\end{bmatrix}
\begin{bmatrix}
f_1(t) \\ f_2(t)
\end{bmatrix}
= \begin{bmatrix}
c_1 f_1(t) \\
c_2 f_2(t) \\
c_1 f_1(t) + c_2 f_2(t) \\
c_1 f_1(t) + c_2 f_2(t) \\
\end{bmatrix}
$$

We are trying to construct the **second** matrix on the left side. This matrix describes which surface forcing data influence each zone. This matrix can be created using just the vertical connection matrix.

In [5]:
def get_size_mat(vert: NDArray) -> NDArray:
    """Construct the matrix describing how the forcing data series
    connect to each other.
    """
    index_rows: list[int] = [i for i, row in enumerate(vert) if row.sum() < 1e-12] # The zone indices of the surface zones. They have no vertical zones above
    composite_rows: list[int] = list(set((i for i in range(vert.shape[0]))).difference(index_rows))

    mat: NDArray = np.zeros((vert.shape[0], len(index_rows)), dtype=float)
    for i, x_i in enumerate(index_rows):
        mat[x_i, i] = 1    

    for c_i in composite_rows:
        mat[c_i] = sum([mat[i] for i, row_i in enumerate(vert[c_i]) if row_i != 0])

    return mat

# Mixing forcing data

This section outlines the methods for constructing the forcing data for a hydrologic model. In a hydrologic model, you may have multiple series of data that you would like to use. In your model, you may have a hillslope and a riparian zone, but you will need to combine these data series in some zone, and this means that you will need some way to weight and combine the precipitation, PET, and temperature.

## Precipitation and PET
When combining precipitation and PET, we are dealing with real physical quantities, so you can just take a weighted average. If you have a hillslope that takes up 70% of the catchment and the riparian zone that takes up 30%, then combining these two series is just:
$$
0.7 x + 0.3 y = z
$$

If you have 2 hillslopes, with the previous hillslope taking up 50% of the entire watershed, then you can combine these two series just by multiplying by another 0.5:
$$
0.5 (0.7 x + 0.3 y) = z
$$

So, the matrix described above is all that is required to combine the series.

## Temperature
Temperature cannot just be scaled by the proportions within a watershed. For example, in a watershed with 50% hillslope and 50% riparian zone, the total precipitation entering the hillslope can be scaled by 0.5, but that makes no sense for temperature. Instead, you will need to **normalize** the scaling. 

One example of where scaling doesn't work is the example above with 2 hillslopes. If the soil zones of the hillslope and riparian zone combine... IDK I'm going to finish writing this later. Each row in the modified matrix is:
$$
\mathbf{F}^{temp}_i = \frac{\mathbf{F}_i}{\sum{\mathbf{F}_i}}
$$

In [6]:
def get_forc_mat(sizes: ArrayLike, conn_mat: NDArray, relative: bool = False) -> NDArray:
    s_arr: NDArray = np.array(sizes)
    mat: NDArray = conn_mat @ np.diag(s_arr)
    if relative:
        return np.divide(mat, mat.sum(axis=1), axis=0)
    else:
        return mat

In [7]:
sizes: list[float] = [0.7, 0.3]
get_forc_mat(sizes, get_size_mat(vert))

array([[0.7, 0. ],
       [0. , 0.3],
       [0.7, 0.3],
       [0.7, 0.3]])

# Example: Two hillslope model

In [8]:
hs_1 = Hillslope([
    Layer([Zone(), Zone()]),
    Layer([Zone(), Zone()]),
    Layer([Zone(), Zone()]),
])

hs_2 = Hillslope([
    Layer([Zone(), Zone()]),
    Layer([Zone()]),
    Layer([Zone()]),
    Layer([Zone()]),
])

sizes: list[float] = [0.2, 0.4, 0.1, 0.3]
# sizes: list[float] = [0.6, 0.4]
model: Model = Model([hs_1, hs_2])
# model: Model = Model([hs_1])
lat: NDArray = model.get_lat_mat()
print(f"Lateral matrix: \n{lat}")
vert: NDArray = model.get_vert_mat()
print(f"Vertical matrix: \n{vert}")
size_mat: NDArray = model.get_size_mat()
print(f"Size matrix: \n{size_mat}")
precip_mat: NDArray = model.get_forc_mat(sizes)
print(f"Precipitation: \n{precip_mat}")
pet_mat: NDArray = model.get_forc_mat(sizes)
print(f"PET: \n{pet_mat}")
temp_mat: NDArray = model.get_forc_mat(sizes, relative=True)
print(f"Temperature: \n{temp_mat}")

Lateral matrix: 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Vertical matrix: 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Size matrix: 
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 1.]
 [0. 0. 1. 1.]
 [0. 0.

# Conclusions
This notebook successfully demonstrates how the matrices describing how the connection matrices were created.